In [1]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.2, max_tokens=300, model_name="gpt-3.5-turbo")

question = "대한민국의 수도는 어디입니까?"

llm.invoke(question)

대한민국의 수도는 어디입니까?
